In [133]:
#!import ./load.ipynb

In [134]:
using System.Globalization;

var sstate = SessionState.GetThreadState();

SessionState.Current.CurrentCulture = CultureInfo.GetCultureInfo("fr-fr");
WriteLine($"Main thread {SessionState.Current.CurrentCulture}");

await Task.Run(() => {
    using (LogTimer.StartThreadInfo("Test Thread"))
    using (sstate?.AttachThread()) {
        WriteLine($"Task thread {SessionState.Current.CurrentCulture}");
    }
});

Main thread fr-FR
Task thread fr-FR


In [135]:
SessionState.Current.CurrentCulture = CultureInfo.GetCultureInfo("fr-fr");
WriteLine($"Main thread {SessionState.Current.CurrentCulture}");

await Concurrent.RunTask(() => {
    WriteLine($"Task thread {SessionState.Current.CurrentCulture}");
});

Main thread fr-FR
Task thread fr-FR


In [136]:
var collection = new BlockingCollection<int>();

var tasks = new Task[3];

tasks[0] = Concurrent.RunTask(() => {
    for (int i = 0; i<3; ++i)
        collection.Add(i);
});
tasks[1] = Concurrent.RunTask(() => {
    for (int i = 0; i<3; ++i)
        collection.Add(10+i);
});
tasks[2] = Concurrent.RunTask(() => {
    for (int i = 0; i<3; ++i)
        collection.Add(20+i);
});
Concurrent.RunTask(() => {
    Task.WaitAll(tasks);
    collection.CompleteAdding();
}).Wait();

var arr = new List<int>();

foreach (int i in collection.GetConsumingEnumerable())
    arr.Add(i);

WriteLine(arr.Select(i => i.ToString()).Join(", "));

20, 21, 22, 0, 1, 2, 10, 11, 12


In [137]:
using Soneta.Towary;

var collection = new BlockingCollection<Session>();

var kody = new List<string>();

using (Session session = Login.CreateSession(false, false, "Producer Session")) {
    using (var transaction = session.Logout(true)) {
        foreach (var towar in session.GetTowary().Towary.WgKodu.GetNextRows(null, 3, out _).Cast<Towar>()) {
            kody.Add(towar.Kod);
            WriteLine($"Towar reset: {towar.Kod}");
            towar.NumerKatalogowy = "";
        }
        transaction.Commit();
    }
    session.Save();
}

WriteTowary();

void WriteTowary() {
    using Session session = Login.CreateSession(false, false, "Write Log");
    foreach (var towar in session.GetTowary().Towary.WgKodu.GetNextRows(null, 3, out _).Cast<Towar>()) {
        WriteLine($"Towar: {towar.Kod}, numer katalogowy: '{towar.NumerKatalogowy}'");
    }
}

Towar reset: BIKINI
Towar reset: BUT_NAR
Towar reset: BUT_NAR_42
Towar: BIKINI, numer katalogowy: ''
Towar: BUT_NAR, numer katalogowy: ''
Towar: BUT_NAR_42, numer katalogowy: ''


In [138]:
void BuildSession(string kod) {
    using Session session = Login.CreateSession(false, false, "Producer Session");
    
    using (var transaction = session.Logout(true)) {
        var towar = session.GetTowary().Towary.WgKodu[kod];
        
        WriteLine($"Task Towar: {towar.Kod} - '{towar.NumerKatalogowy}'");
        towar.NumerKatalogowy = "Numer: "+kod.ToLower();
        
        transaction.Commit();
    }
    
    session.Save();
}

var tasks = new[] {
    Concurrent.RunTask(() => BuildSession(kody[0])),
    Concurrent.RunTask(() => BuildSession(kody[1])),
    Concurrent.RunTask(() => BuildSession(kody[2]))
};
await Task.WhenAll(tasks);

WriteTowary();

Task Towar: BUT_NAR - ''
Task Towar: BIKINI - ''
Task Towar: BUT_NAR_42 - ''
Towar: BIKINI, numer katalogowy: 'Numer: bikini'
Towar: BUT_NAR, numer katalogowy: 'Numer: but_nar'
Towar: BUT_NAR_42, numer katalogowy: 'Numer: but_nar_42'


In [139]:
Session BuildSessionResult(string kod) {
    using Session session = Login.CreateSession(false, false, "Producer Session");
    
    using (var transaction = session.Logout(true)) {
        var towar = session.GetTowary().Towary.WgKodu[kod];
        
        WriteLine($"Task Towar: {towar.Kod} - '{towar.NumerKatalogowy}'");
        towar.NumerKatalogowy = "Numer: "+kod.ToLower();
        
        transaction.Commit();
    }
    
    return session;
}

var sessions = await Task.WhenAll([
    Concurrent.RunTask<Session>(() => BuildSessionResult(kody[0])),
    Concurrent.RunTask<Session>(() => BuildSessionResult(kody[1])),
    Concurrent.RunTask<Session>(() => BuildSessionResult(kody[2]))
]);

WriteLine("Before save");
WriteTowary();

using (var dbTransaction = Database.Logout()) {
    foreach (Session session in sessions)
        session.Save();

    dbTransaction.Commit();
}

WriteLine("After save");
WriteTowary();

Task Towar: BUT_NAR - 'Numer: but_nar'
Task Towar: BIKINI - 'Numer: bikini'
Task Towar: BUT_NAR_42 - 'Numer: but_nar_42'
Before save
Towar: BIKINI, numer katalogowy: 'Numer: bikini'
Towar: BUT_NAR, numer katalogowy: 'Numer: but_nar'
Towar: BUT_NAR_42, numer katalogowy: 'Numer: but_nar_42'
After save
Towar: BIKINI, numer katalogowy: 'Numer: bikini'
Towar: BUT_NAR, numer katalogowy: 'Numer: but_nar'
Towar: BUT_NAR_42, numer katalogowy: 'Numer: but_nar_42'
